# Hierarchical Density-Based Spatial Clustering of Applications 


https://github.com/scikit-learn-contrib/hdbscan


https://towardsdatascience.com/understanding-hdbscan-and-density-based-clustering-121dbee1320e



In [1]:

import hdbscan
# from sklearn.datasets import make_blobs

# data, _ = make_blobs(1000)
# print(data)
# clusterer = hdbscan.RobustSingleLinkage(cut=0.125, k=7)
# cluster_labels = clusterer.fit_predict(data)
# hierarchy = clusterer.cluster_hierarchy_
# alt_labels = hierarchy.get_clusters(0.500, 5)
# hierarchy.plot()
# tfidf vectorizer

In [2]:
# Final Code
#Step 1 vectorize data

import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy          as np
import pandas         as pd
import datetime
import time
import sys
import re
from gensim.parsing.preprocessing import remove_stopwords, split_alphanum, strip_punctuation, strip_short, strip_multiple_whitespaces, strip_non_alphanum, strip_numeric


df= pd.read_csv('../data/cluster-sub-cluster.csv')
df['text'] = df['text'].fillna('').apply(str)


In [3]:
#preprocessing
def clean(text):
    text = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', text, flags=re.MULTILINE) # urls
    text = re.sub(r'[\s]{0,1}([@#][\w_-]+)', '', text, flags=re.MULTILINE) # @mentions and #hashtags
    text = text.lower() # lowercase all text to use an uncased bert model
    text = text.strip(',.-_ ')
    text = split_alphanum(text.lower())
    text = remove_stopwords(text.lower())
    text = strip_non_alphanum(text.lower())
    text = strip_short(text.lower())
    text = strip_multiple_whitespaces(text.lower())
    text = re.sub('[\d]+', '', text, flags=re.MULTILINE)
    return text


# Change name of text column
df['clean_text'] = df['text'].apply(clean)
# remove empty text data due to the previous cleaning step
df['clean_text'] = df['clean_text'].replace('', np.nan)
df['clean_text'] = df['clean_text'].replace('nan', np.nan)
df = df.dropna(subset=['clean_text'])

print(df)

                                                   text          published_at  \
0     @JackPosobiec How about sending the fucking co...  2020-06-26T11:37:39Z   
1     @bennyjohnson @EricTrump Out of all of those d...  2020-06-26T11:36:37Z   
2     Deep state on the run, POTUS trolling the deep...  2020-06-26T11:29:59Z   
3     We DO KNOW someone behind BLM want to DESTROY ...  2020-06-26T11:27:04Z   
4     @GOPLeader We DO KNOW someone behind BLM want ...  2020-06-26T11:26:58Z   
...                                                 ...                   ...   
5987  @ZoomerNeolib The Dem will be Biden’s vp. Like...  2020-06-21T23:22:38Z   
5988  @mike10010100 @VaushV Rachel Maddow, Anderson ...  2020-06-21T23:18:27Z   
5989  The anti racists TV show Watchmen got 9.6 mill...  2020-06-21T23:13:20Z   
5990  @SaintMystic I really wanted it to be Kamala a...  2020-06-21T23:11:24Z   
5991  Kamala Harris' record makes her 'too comfortab...  2020-06-21T23:08:29Z   

     platform  engagement  

In [4]:
# %%timeit
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
# print(vectorizer.get_feature_names())
# print(vectorizer.vocabulary_)
data = X.toarray()
print(data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from timeit import default_timer as timer
start=timer()

clusterer = hdbscan.RobustSingleLinkage(cut=0.125, k=7)
cluster_labels = clusterer.fit_predict(data)
hierarchy = clusterer.cluster_hierarchy_
alt_labels = hierarchy.get_clusters(0.500, 5)

print("Total processing time in seconds")
print(timer()-start)

In [21]:

df1=pd.DataFrame()
df1['lbl1']=list(alt_labels)
df1['lbl2']=list(clusterer.labels_)
print(df1[df1['lbl1']!=df1['lbl2']])


      lbl1  lbl2
76      12    14
104     12    14
106     12    14
153     12    14
179      4     7
201     12    14
205     12    14
226     12    14
239     12    14
264     12    14
271     12    14
278     12    14
323     12    14
330      3     6
371      5     8
389     12    14
404      7     9
417     12    14
418     12    14
436     12    14
445     13    15
448      1     4
450      1     4
451      1     4
452      1     4
459      4     7
464      0    -1
465      0     3
474      7     9
506     12    -1
515      1     4
516      1     4
517      1     4
519      1     4
520      1     4
528     12    -1
541      7     9
544     12    14
550     12    14
554     12    14
571      4     7
579     16    17
580     16    17
582     16    17
590     19    20
605     12    14
616     12    14
625      7     9
645     12    14
649     12    14
656     16    17
669     12    14
673      1     4
674      1     4
675      1     4
676      1     4
677      1     4
680      1    

In [7]:
df['hdbscan_label']=list(alt_labels)

In [12]:
labeled_df=df[['clean_text','label', 'hdbscan_label']]
filtered_df=labeled_df[labeled_df['hdbscan_label'] !=-1]
filtered_df.groupby(['hdbscan_label']).count().sort_values(['label'], ascending=False)
# filtered_df.sort_values(['label'])

,clean_text,label
hdbscan_label,,
0,174,174
8,79,79
12,61,61
1,38,38
9,29,29
27,21,21
26,20,20
22,20,20
18,16,16


In [15]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', filtered_df.shape[0]+1)
filtered_df.sort_values(['hdbscan_label'])

# df = pandas.read_csv("data.csv")
# pandas.set_option('display.max_rows', df.shape[0]+1)
# print(df)

,clean_text,label,hdbscan_label
4,know blm want destroy great country democrats use blm campaign issue regain power blm communism culture revolution maga maga positive away blm lefts covid,label,0
3709,know blm want destroy great country blm communism culture revolution maga maga positive blm negative,MAGA posting,0
3694,know purpose blm know blm want destroy great country blm communism culture revolution maga maga positive blm negative,label,0
3605,know blm want destroy great country blm communism culture revolution maga maga positive blm negative,MAGA posting,0
3604,know blm want destroy great country blm communism culture revolution maga maga positive blm negative,MAGA posting,0
3603,know blm want destroy great country blm communism culture revolution maga maga positive blm negative,MAGA posting,0
3503,know purpose blm know blm want destroy great country blm communism culture revolution maga maga positive blm negative,label,0
3502,know purpose blm know blm want destroy great country blm communism culture revolution maga maga positive blm negative,label,0
3501,know purpose blm know blm want destroy great country blm communism culture revolution maga maga positive blm negative,label,0
3500,know purpose blm know blm want destroy great country blm communism culture revolution maga maga positive blm negative,label,0


In [20]:
# Store in CSV
labeled_df=df[['text', 'clean_text','label', 'color', 'hdbscan_label']]
filtered_df=labeled_df[labeled_df['hdbscan_label'] !=-1]


filtered_df.to_csv('../data/cluster-sub-cluster-with-new-labels.csv', encoding='utf-8', index=True)

# # A way to return the clusters.
# a = np.array(clusterer.labels_)
# unique, counts = np.unique(a, return_counts=True)
# x = dict(zip(unique, counts))

# {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}

In [ ]:
# A way to return the clusters.
# a = np.array(clusterer.labels_)
# unique, counts = np.unique(a, return_counts=True)
# x = dict(zip(unique, counts))

# {k: v for k, v in sorted(x.items(), key=lambda item: item[1])}

# labels = np.array(clusterer.labels_)
# cluster_index_1 = np.where(labels == 0)
# cluster_index_2 = np.where(labels == 1)
# cluster_index_3 = np.where(labels == 2)

# print(cluster_index_1)
# for x in np.nditer(cluster_index_1):
#     print(x)
#     print(df['text'][x])
#     print("\n")

In [ ]:
%timeit sum(range(100))

In [ ]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j